In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2021-03-05 10:46:51,595 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\access.log


In [3]:
%load_ext autoreload 
%autoreload 2
from virasana.integracao.conformidade_alchemy import Conformidade
from virasana.scripts.conformidadeupdate import completa_conformidade, preenche_isocode, preenche_bbox
engine = create_engine(SQL_URI)
completa_conformidade(db, engine, 200)
preenche_isocode(db, engine, 200)
preenche_bbox(db, engine, 200)

2021-03-05 10:50:23,517 ajna         INFO     2000 análises de conformidade complementadas em 190.89916491508484 segundos.0.09544958245754243 por registro
2021-03-05 10:52:04,252 ajna         INFO     1567 isocode preenchidos em 100.01600098609924 segundos.0.06382642053994847 por registro


# Checagens de conformidade

In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
rs = session.execute('SELECT count(*) FROM dbmercante.ajna_conformidade;')

In [6]:
rs.scalar()

118949

In [10]:
SQL_CONFORMIDADE = '''
select cod_recinto, count(*), avg(height), std(height),
SUM(if(height < 700, 1, 0)) / count(*) * 100 as height_baixo,
avg(ratio), stddev(ratio),  min(ratio), max(ratio),
 SUM(if(ratio < 1.9, 1, 0)) / count(ratio) * 100 as ratio_baixo
 avg(laplacian)
 from ajna_conformidade group by cod_recinto
 '''
df = pd.read_sql(SQL_CONFORMIDADE, engine)

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'avg(laplacian)\n from ajna_conformidade group by cod_recinto' at line 5")
[SQL: 
select cod_recinto, count(*), avg(height), std(height),
SUM(if(height < 700, 1, 0)) / count(*) * 100 as height_baixo,
avg(ratio), stddev(ratio),  min(ratio), max(ratio),
 SUM(if(ratio < 1.9, 1, 0)) / count(ratio) * 100 as ratio_baixo
 avg(laplacian)
 from ajna_conformidade group by cod_recinto
 ]
(Background on this error at: http://sqlalche.me/e/13/f405)

In [ ]:
df